In [ ]:
!pip install opencv-python

In [146]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from threading import Thread

In [151]:
def get_road_marks(frame, drawto):
  f = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  retval, thresh = cv2.threshold(f, 180, 255, cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE , cv2.CHAIN_APPROX_NONE)
  larges = [cnt for cnt in contours if cv2.contourArea(cnt) < 1250]
  subf = drawto
  cv2.drawContours(subf, larges, -1, (0,0,255), 1, cv2.LINE_4, hierarchy, 1 )
  cv2_imshow(subf)

In [152]:
def process(frame, frameid):
  fg_mask = backSub.apply(frame)
  frameid += 1
  if frameid > 5:
    get_road_marks(backSub.getBackgroundImage(), frame.copy())


  #cv2_imshow(backSub.getBackgroundImage())
  contours, hierarchy = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  retval, mask_thresh = cv2.threshold( fg_mask, 180, 255, cv2.THRESH_BINARY)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
  # Apply erosion
  mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)

  large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]
  frame_out = frame
  for cnt in large_contours:
      x, y, w, h = cv2.boundingRect(cnt)
      frame_out = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 200), 3)

  # отображаем результат
  cv2_imshow(frame_out)

In [ ]:
cap = cv2.VideoCapture('./inp.mp4')
cap.set(cv2.CAP_PROP_FPS, 10)
backSub = cv2.createBackgroundSubtractorMOG2()
if not cap.isOpened():
    print("Error opening video file")

rets = 0
frameid = 0
while cap.isOpened():
      print(f'{frameid}')
      # Захват кадр за кадром
      ret, frame = cap.read()

      if ret:
          rets = 0
          frameid += 1
          process(frame, frameid)
          #frame = cv2.resize(frame, (320, 240))

      else:
        if rets > 10:
          break
        rets += 1